# Filtro de spam
## Carregamento do dataset

In [4]:
import pandas as pd

# Carregar o dataset Boston Housing
df = pd.read_csv('spam_assassin.csv')

df.head()

,text,target
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0


## Pré-processamento - Limpeza - Normalização

### Conceitos básicos

#### Tokenização
Tokenização é o processo de dividir um texto em unidades menores, chamadas tokens, que podem ser palavras, frases, ou até caracteres individuais, dependendo do contexto e objetivo da análise. Em processamento de linguagem natural (PLN), a tokenização é uma etapa essencial para transformar texto bruto em dados estruturados, permitindo que as palavras ou frases sejam analisadas separadamente. A tokenização facilita operações subsequentes, como análise de frequência, remoção de stopwords, e representação vetorial, ajudando algoritmos de aprendizado de máquina a entender e processar texto de maneira mais eficiente.

#### Stopwords
Stopwords são palavras comuns em um idioma que, por si só, geralmente não carregam muito significado para a análise de texto, como "e", "de", "para" e "o" em português, ou "and", "the", "of" e "in" em inglês. No processamento de linguagem natural, as stopwords são frequentemente removidas dos dados para reduzir o ruído e melhorar o desempenho dos algoritmos, concentrando a análise nas palavras que carregam mais informações relevantes. Excluir essas palavras ajuda a simplificar o texto e diminuir a dimensionalidade dos dados, facilitando tarefas como classificação de texto, análise de sentimento e recuperação de informações.


#### Stemming
Stemming é o processo de reduzir palavras às suas formas base ou radicais, removendo sufixos e outras partes flexionadas. Essa técnica, usada em processamento de linguagem natural, simplifica o texto ao agrupar variações morfológicas de uma palavra (como plurais, tempos verbais, etc.) em um único termo comum, sem considerar o contexto completo. Ferramentas de stemming, como o PorterStemmer, ajudam a normalizar o vocabulário, diminuindo a dimensionalidade e a complexidade dos dados textuais, o que facilita tarefas como classificação de texto e recuperação de informação.


In [2]:
!pip install nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

# Baixar stopwords e tokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# o dataset está em inglês, então serão consideradas as 
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def remover_cabecalhos(email_texto):
    # Remove todas as linhas de cabeçalhos que começam com um padrão de cabeçalho típico de email
    # "^" indica o início da linha e "(?i)" torna a regex case-insensitive
    email_sem_cabecalhos = re.sub(r"(?i)^(From|To|Date|Subject|Reply-To|Message-ID|MIME-Version|Content-Type|Content-Transfer-Encoding|Received|X-\w+):.*\n?", '', email_texto, flags=re.MULTILINE)
    
    # Remove possíveis linhas em branco após a remoção dos cabeçalhos
    email_sem_cabecalhos = email_sem_cabecalhos.strip()
    
    return email_sem_cabecalhos

def limpar_texto(texto):
    # Converte o texto para minúsculas
    texto = texto.lower()

    # Remove cabeçalhos de e-mail
    texto = remover_cabecalhos(texto)
    
    # Remove URLs
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    
    # Remove as pontuações
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    
    # Remova números usando expressões regulares
    texto = re.sub(r'\d+', '', texto)
    
    # Tokenização
    palavras = word_tokenize(texto)
    
    # Remove stopwords e aplica o stemming
    palavras_limpa = [stemmer.stem(palavra) for palavra in palavras if palavra not in stop_words]
    
    # Reconstrue o texto limpo
    texto_limpo = ' '.join(palavras_limpa)
    
    return texto_limpo

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
df['text_clean'] = df['text'].apply(limpar_texto)
df.head()

,text,target,text_clean
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0,ilugadminlinuxi mon jul returnpath ilugadminli...
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1,gortexcitecom mon jun returnpath gortexcitecom...
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1,forkadminxentcom mon jul returnpath forkadminx...
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1,dcmbtamailnetcn mon jun returnpath dcmbtamailn...
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0,ilugadminlinuxi mon aug returnpath ilugadminli...


## Pré-processamento - Transformação

### Representação de texto
#### Bag of Words
A técnica Bag of Words (BoW) é um método simples de representação de texto que transforma documentos em uma matriz de contagem de palavras, ignorando a ordem das palavras e focando na frequência de cada termo. Cada coluna da matriz representa uma palavra única no corpus, e cada célula indica o número de vezes que essa palavra aparece em um determinado documento. Apesar de sua simplicidade, o BoW é eficaz em capturar a presença de termos e é amplamente utilizado em tarefas de processamento de linguagem natural, como classificação de texto e análise de sentimento. No entanto, o BoW não considera contexto ou relações semânticas entre as palavras, limitando sua capacidade de capturar nuances de significado.


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Inicializa o vetorizador Bag of Words
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(df["text_clean"])

# Converte a matriz esparsa para um DataFrame para visualização
df_bow = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df_bow.head()


,aa,aaa,aaaa,aaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaqaaaaia,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacqaaayqb,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaghoawajwa,...,zzzzteanaegroup,zzzzteanaegroupscom,zzzzteanataintorg,zzzzteanayahoogroupscom,zzzzuseperlspamassassintaintorg,zzzzvfmtyfgwtqbhmwzuffranxlydunhsdtqqprryuelttfrpjpoqgeartzjiaj,zzzzvpexofhibogkbjdemkenzxddoadomnkmbplmazkpuaeptcmaewxfeawaaamacgcfvti,zzzzwebdevspamassassintaintorg,zzzzwebnotenet,zzzzyour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### TF-IDF (Term Frequency - Inverse Document Frequency)
A técnica TF-IDF (Term Frequency - Inverse Document Frequency) é uma representação vetorial que pondera cada termo com base em sua frequência no documento e na relevância dentro do corpus, destacando palavras distintivas de cada documento. A frequência de um termo (TF) mede quantas vezes ele aparece no documento, enquanto a frequência inversa de documentos (IDF) reduz o peso de palavras muito comuns no corpus, como artigos ou preposições. Dessa forma, a TF-IDF atribui valores mais altos a palavras que são importantes para um documento específico, mas que não são frequentes no conjunto geral de documentos, o que torna essa técnica especialmente útil para tarefas de recuperação de informação e classificação de texto, onde é importante capturar a relevância de termos específicos.


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializa o vetorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df["text_clean"])

# Converte a matriz esparsa para um DataFrame para visualização
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df_tfidf.head()


,aa,aaa,aaaa,aaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaqaaaaia,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacqaaayqb,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaghoawajwa,...,zzzzteanaegroup,zzzzteanaegroupscom,zzzzteanataintorg,zzzzteanayahoogroupscom,zzzzuseperlspamassassintaintorg,zzzzvfmtyfgwtqbhmwzuffranxlydunhsdtqqprryuelttfrpjpoqgeartzjiaj,zzzzvpexofhibogkbjdemkenzxddoadomnkmbplmazkpuaeptcmaewxfeawaaamacgcfvti,zzzzwebdevspamassassintaintorg,zzzzwebnotenet,zzzzyour
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Classificação

### Antes de tudo, como interpretar os resultados
**Precision, Recall, F1-Score e Support** são métricas de avaliação para modelos de classificação, especialmente úteis em tarefas de classificação binária e multilabel.

- **Precision (Precisão)**: Mede a proporção de predições positivas corretas em relação ao total de predições positivas feitas pelo modelo. Em um contexto de spam, uma alta precisão significa que a maioria dos emails classificados como spam realmente são spam.

- **Recall (Sensibilidade ou Revocação)**: Mede a proporção de predições positivas corretas em relação ao total de positivos reais no conjunto de dados. Em termos de spam, uma alta recall significa que o modelo identificou a maioria dos emails de spam existentes.

- **F1-Score**: É a média harmônica entre precisão e recall, equilibrando as duas métricas. É útil quando há um trade-off entre precisão e recall, especialmente em casos onde ambas são importantes.

- **Support**: Representa o número total de ocorrências de cada classe no conjunto de dados de teste (ex.: o número total de emails de spam e não-spam). Isso é importante para entender o equilíbrio de classes e o desempenho do modelo em cada uma.


### Usando o modelo clássico de Naive Bayes
**Naive Bayes** é um modelo probabilístico baseado no Teorema de Bayes, amplamente utilizado para classificação de textos, como na filtragem de spam. A principal suposição do Naive Bayes é a de que as características (ou palavras, no caso de texto) são independentes entre si, o que significa que a presença de uma palavra não influencia a presença de outra. Essa simplificação torna o Naive Bayes rápido e eficiente, especialmente em problemas de texto, mesmo que essa independência completa raramente seja verdadeira. Existem várias variantes do Naive Bayes, sendo a Multinomial Naive Bayes a mais comum para problemas de classificação de texto, onde a frequência das palavras (ou tokens) é levada em consideração.

In [11]:
# Separação dos dados em 2 conjuntos
from sklearn.model_selection import train_test_split

# Dividindo os dados em treino e teste (80% treino, 20% teste) - bow
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow_matrix, df['target'], test_size=0.2, random_state=42)

# Dividindo os dados em treino e teste (80% treino, 20% teste) - tfidf
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(df_tfidf, df['target'], test_size=0.2, random_state=42)


In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Modelo Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_bow, y_train_bow)

# Previsões e Avaliação
y_pred_nb = nb_model.predict(X_test_bow)
print("Naive Bayes - Acurácia:", accuracy_score(y_test_bow, y_pred_nb))
print("Relatório de Classificação:\n", classification_report(y_test_bow, y_pred_nb))


Naive Bayes - Acurácia: 0.9474137931034483
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96       779
           1       1.00      0.84      0.91       381

    accuracy                           0.95      1160
   macro avg       0.96      0.92      0.94      1160
weighted avg       0.95      0.95      0.95      1160



In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Modelo Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train_tfidf)

# Previsões e Avaliação
y_pred_nb = nb_model.predict(X_test_tfidf)
print("Naive Bayes - Acurácia:", accuracy_score(y_test_tfidf, y_pred_nb))
print("Relatório de Classificação:\n", classification_report(y_test_tfidf, y_pred_nb))

Naive Bayes - Acurácia: 0.8853448275862069
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.85      1.00      0.92       779
           1       1.00      0.65      0.79       381

    accuracy                           0.89      1160
   macro avg       0.93      0.83      0.85      1160
weighted avg       0.90      0.89      0.88      1160



### Regressão logística
**Regressão Logística** é um modelo de classificação linear que usa a função logística (ou sigmoid) para mapear a soma ponderada das características de entrada para uma probabilidade entre 0 e 1. Diferente do Naive Bayes, a Regressão Logística não assume independência entre as características, o que pode tornar o modelo mais flexível em cenários onde as variáveis têm alguma correlação. É comumente usado em problemas binários, como detectar se um email é spam ou não. A Regressão Logística otimiza um conjunto de pesos para cada característica, ajustando o modelo para maximizar a probabilidade dos dados observados, fornecendo uma classificação robusta para tarefas de texto e outros tipos de dados.


In [14]:
from sklearn.linear_model import LogisticRegression

# Modelo de Regressão Logística
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_bow, y_train_bow)

# Previsões e Avaliação
y_pred_lr = lr_model.predict(X_test_bow)
print("Regressão Logística - Acurácia:", accuracy_score(y_test_bow, y_pred_lr))
print("Relatório de Classificação:\n", classification_report(y_test_bow, y_pred_lr))


Regressão Logística - Acurácia: 0.996551724137931
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       779
           1       1.00      0.99      0.99       381

    accuracy                           1.00      1160
   macro avg       1.00      0.99      1.00      1160
weighted avg       1.00      1.00      1.00      1160



In [15]:
from sklearn.linear_model import LogisticRegression

# Modelo de Regressão Logística
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_tfidf, y_train_tfidf)

# Previsões e Avaliação
y_pred_lr = lr_model.predict(X_test_bow)
print("Regressão Logística - Acurácia:", accuracy_score(y_test_tfidf, y_pred_lr))
print("Relatório de Classificação:\n", classification_report(y_test_tfidf, y_pred_lr))


Regressão Logística - Acurácia: 0.8974137931034483
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.87      1.00      0.93       779
           1       1.00      0.69      0.81       381

    accuracy                           0.90      1160
   macro avg       0.93      0.84      0.87      1160
weighted avg       0.91      0.90      0.89      1160



/opt/conda/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


### MLP (Multi Layer Perceptron)
**Classificação com MLP (Perceptron Multicamada)** usa uma rede neural artificial com múltiplas camadas de neurônios para capturar padrões complexos nos dados, incluindo relações não lineares entre as características. Em uma MLP, a informação passa pela camada de entrada, uma ou mais camadas ocultas e a camada de saída, onde cada neurônio aplica uma função de ativação para ajudar a rede a aprender padrões de classificação. A MLP é treinada usando algoritmos de retropropagação, ajustando os pesos das conexões para minimizar o erro de predição. Esse modelo é particularmente útil para dados mais complexos, mas exige mais tempo de treinamento e processamento, além de necessitar de uma quantidade maior de dados em comparação com modelos lineares como Naive Bayes e Regressão Logística.

In [16]:
from sklearn.neural_network import MLPClassifier

# Modelo de MLP
mlp_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42)
mlp_model.fit(X_train_bow, y_train_bow)

# Previsões e Avaliação
y_pred_mlp = mlp_model.predict(X_test_bow)
print("MLP - Acurácia:", accuracy_score(y_test_bow, y_pred_mlp))
print("Relatório de Classificação:\n", classification_report(y_test_bow, y_pred_mlp))


MLP - Acurácia: 0.9974137931034482
Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       779
           1       1.00      0.99      1.00       381

    accuracy                           1.00      1160
   macro avg       1.00      1.00      1.00      1160
weighted avg       1.00      1.00      1.00      1160



In [17]:
from sklearn.neural_network import MLPClassifier

# Modelo de MLP
mlp_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42)
mlp_model.fit(X_train_tfidf, y_train_tfidf)

# Previsões e Avaliação
y_pred_mlp = mlp_model.predict(X_test_tfidf)
print("MLP - Acurácia:", accuracy_score(y_test_tfidf, y_pred_mlp))
print("Relatório de Classificação:\n", classification_report(y_test_tfidf, y_pred_mlp))


MLP - Acurácia: 0.9974137931034482
Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       779
           1       1.00      0.99      1.00       381

    accuracy                           1.00      1160
   macro avg       1.00      1.00      1.00      1160
weighted avg       1.00      1.00      1.00      1160

